# Bulk RNASeq Demonstration

The data I am using are comparative transcriptomes of soybeans grown at either ambient or elevated O3 levels. Each condition was done in triplicate. The paper that these samples come from (which also serves as a great background reading on RNA-seq) can be found here: [Yendrek et al. 2012](https://bmcresnotes.biomedcentral.com/articles/10.1186/1756-0500-5-506).

### Download data

In [2]:
%%time
!mkdir -p ../data/SRA_data
!fasterq-dump SRR391535 --outdir ../data/SRA_data
!fasterq-dump SRR391536 --outdir ../data/SRA_data
!fasterq-dump SRR391537 --outdir ../data/SRA_data
!fasterq-dump SRR391538 --outdir ../data/SRA_data
!fasterq-dump SRR391539 --outdir ../data/SRA_data
!fasterq-dump SRR391541 --outdir ../data/SRA_data

spots read      : 31,889,531
reads read      : 31,889,531
reads written   : 31,889,531
spots read      : 16,862,414
reads read      : 16,862,414
reads written   : 16,862,414
spots read      : 17,575,844
reads read      : 17,575,844
reads written   : 17,575,844
spots read      : 28,554,551
reads read      : 28,554,551
reads written   : 28,554,551
2023-11-05T02:54:41 fasterq-dump.3.0.7 err: insp_get_file_size( 'https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR391539/SRR391539', remotely = YES ) -> RC(rcKrypto,rcFile,rcOpening,rcConnection,rcFailed)
spots read      : 37,605,167
reads read      : 37,605,167
reads written   : 37,605,167
spots read      : 36,408,402
reads read      : 36,408,402
reads written   : 36,408,402
CPU times: user 6min 12s, sys: 28.2 s, total: 6min 40s
Wall time: 1h 24min 21s


### Trim the data

In [13]:
%%time
!for i in ../data/SRA_data/*fastq; do \
    trimmomatic \
        SE \
        -phred33 \
        $i \
        ${i::-6}_trimmed.fastq \
        SLIDINGWINDOW:4:35 \
        MINLEN:45; done

TrimmomaticSE: Started with arguments:
 -phred33 ../data/SRA_data/SRR391535.fastq ../data/SRA_data/SRR391535_trimmed.fastq SLIDINGWINDOW:4:35 MINLEN:45
Automatically using 4 threads
Input Reads: 31889531 Surviving: 21667114 (67.94%) Dropped: 10222417 (32.06%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -phred33 ../data/SRA_data/SRR391536.fastq ../data/SRA_data/SRR391536_trimmed.fastq SLIDINGWINDOW:4:35 MINLEN:45
Automatically using 4 threads
Input Reads: 16862414 Surviving: 12373877 (73.38%) Dropped: 4488537 (26.62%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -phred33 ../data/SRA_data/SRR391537.fastq ../data/SRA_data/SRR391537_trimmed.fastq SLIDINGWINDOW:4:35 MINLEN:45
Automatically using 4 threads
Input Reads: 17575844 Surviving: 12988522 (73.90%) Dropped: 4587322 (26.10%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -phred33 ../data/SRA_data/SRR391538.fastq ../data/SRA_data/SRR391538

In [14]:
!mv ../data/SRA_data/*trimmed.fastq ../data/
!rm -rf ../data/SRA_data/

### Run fastqc to visualize the read quality

This takes very long, so I will only do this for one of the files. Normally, I would run this on the cloud or an HPC cluster, but since I am doing this on my own, and I do not want to pay for the cloud and do not have access to a cluster.

In [10]:
!fastqc \
    data/trimmed_SRR391535.fastq \
    -t 4 \
    -o data/

text/plain
Started analysis of trimmed_SRR391535.fastq
^C


![This is one example of the QC plots](path-to-image)

### Download reference soybean genome and create index

In [5]:
!curl \
    --cookie jgi_session=/api/sessions/7639b96d5266ccacdff6d8cce6c0b33a \
    --output download.20231104.173936.zip \
    -d "{\"ids\":{\"Phytozome-275\":[\"53112af049607a1be0055a33\",\"53112aec49607a1be0055a29\"]}}" \
    -H "Content-Type: application/json" https://files.jgi.doe.gov/filedownload/
!unzip download.20231104.173936.zip
!rm download.20231104.173936.zip
!rm Download_6*.csv
!mv Phytozome/PhytozomeV10/Gmax/ data/
!rm -rf Phytozome/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  283M    0  283M  100    81  7520k      2  0:00:40  0:00:38  0:00:02 9101k
Archive:  download.20231104.173936.zip
  inflating: Download_668976_File_Manifest.csv  
  inflating: Phytozome/PhytozomeV10/Gmax/assembly/Gmax_275_v2.0.fa.gz  
  inflating: Phytozome/PhytozomeV10/Gmax/annotation/Gmax_275_Wm82.a2.v1.gene_exons.gff3.gz  
rm: cannot remove 'Download_668975_File_Manifest.csv': No such file or directory


In [14]:
!gunzip data/Gmax/assembly/Gmax_275_v2.0.fa.gz
!gunzip data/Gmax/annotation/Gmax_275_Wm82.a2.v1.gene_exons.gff3.gz

gzip: data/Gmax/assembly/Gmax_275_v2.0.fa.gz: No such file or directory


In [21]:
!mv data/Gmax/assembly/Gmax_275_v2.0.fa data/Gmax/
!mv data/Gmax/annotation/Gmax_275_Wm82.a2.v1.gene_exons.gff3 data/Gmax/
!rm -rf data/Gmax/assembly/
!rm -rf data/Gmax/annotation/

In [29]:
!mkdir -p data/genome
!STAR \
    --runMode genomeGenerate \
    --genomeDir data/genome \
    --genomeFastaFiles data/Gmax/Gmax_275_v2.0.fa \
    --sjdbGTFfile data/Gmax/Gmax_275_Wm82.a2.v1.gene_exons.gff3 \
    --sjdbGTFtagExonParentTranscript Parent \
    --sjdbOverhang 100 \
    --limitGenomeGenerateRAM 10000000000 \
    --genomeSAindexNbases 9 \
    --genomeSAsparseD 4 \
    --runThreadN 4

Nov 05 00:23:17 ..... started STAR run
Nov 05 00:23:17 ... starting to generate Genome files
Nov 05 00:24:16 ... starting to sort Suffix Array. This may take a long time...
Nov 05 00:24:23 ... sorting Suffix Array chunks and saving them to disk...
Nov 05 00:32:19 ... loading chunks from disk, packing SA...
Nov 05 00:32:45 ... finished generating suffix array
Nov 05 00:32:45 ... generating Suffix Array index
Nov 05 00:32:48 ... completed Suffix Array index
Nov 05 00:32:48 ..... processing annotations GTF
Nov 05 00:34:32 ..... inserting junctions into the genome indices
Nov 05 00:43:26 ... writing Genome to disk ...
Nov 05 00:43:30 ... writing Suffix Array to disk ...
Nov 05 00:43:46 ... writing SAindex to disk
Nov 05 00:43:47 ..... finished successfully


### Align trimmed reads to reference

In [16]:
!for i in ../data/*trimmed.fastq; do \
    STAR \
        --genomeDir ../data/genome \
        --readFilesIn $i \
        --runThreadN 4 \
        --outSAMtype BAM SortedByCoordinate \
        --outFileNamePrefix ${i::-14}; done

Nov 05 04:02:21 ..... started STAR run
Nov 05 04:02:21 ..... loading genome
Nov 05 04:02:48 ..... started mapping
Nov 05 04:38:03 ..... started sorting BAM
Nov 05 04:39:51 ..... finished successfully
Nov 05 04:39:52 ..... started STAR run
Nov 05 04:39:53 ..... loading genome
Nov 05 04:40:14 ..... started mapping
Nov 05 04:58:54 ..... started sorting BAM
Nov 05 04:59:55 ..... finished successfully
Nov 05 04:59:56 ..... started STAR run
Nov 05 04:59:56 ..... loading genome
Nov 05 05:00:11 ..... started mapping
Nov 05 05:19:45 ..... started sorting BAM
Nov 05 05:20:48 ..... finished successfully
Nov 05 05:20:49 ..... started STAR run
Nov 05 05:20:49 ..... loading genome
Nov 05 05:21:04 ..... started mapping
Nov 05 05:48:57 ..... started sorting BAM
Nov 05 05:50:27 ..... finished successfully
Nov 05 05:50:28 ..... started STAR run
Nov 05 05:50:28 ..... loading genome
Nov 05 05:50:43 ..... started mapping
Nov 05 06:24:21 ..... started sorting BAM
Nov 05 06:26:21 ..... finished successfully


### Convert BAM files to raw counts

In [ ]:
%%time
!for i in ../data/*bam; do \
    htseq-count \
        -s no \
        -r pos \
        -t exon \
        -i pacid \
        -f bam \
        $i \
        ../data/Gmax/Gmax_275_Wm82.a2.v1.gene_exons.gff3 > ${i:0:9}-output_basename.counts; done

## Start working in R for analysis

### Setup and import data

In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("BiocManager", repos = "http://cran.us.r-project.org")
library(BiocManager)
BiocManager::install("DESeq2")
library("DESeq2")

R[write to console]: trying URL 'http://cran.us.r-project.org/src/contrib/BiocManager_1.30.22.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 582690 bytes (569 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write 

x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   `libpng-config --cflags` -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1642327566152/work=/usr/local/src/conda/r-base-4.1.2 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c dummy.c -o dummy.o
x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   `libpng-config --cflags` -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_16

installing to /opt/conda/lib/R/library/00LOCK-png/00new/png/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (png)
* installing *source* package ‘GenomeInfoDbData’ ...
** using staged installation
** data
** inst
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (GenomeInfoDbData)
* installing *source* package ‘zlibbioc’ ...
** using staged installation


configure: creating ./config.status
config.status: creating src/Makevars


** libs


...

** libs


x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG -DLIBXML -I/opt/conda/include/libxml2 -I/opt/conda/include -DUSE_EXTERNAL_SUBSET=1 -DROOT_HAS_DTD_NODE=1 -DDUMP_WITH_ENCODING=1 -DUSE_XML_VERSION_H=1 -DXML_ELEMENT_ETYPE=1 -DXML_ATTRIBUTE_ATYPE=1 -DNO_XML_HASH_SCANNER_RETURN=1 -DLIBXML_NAMESPACE_HAS_CONTEXT=1 -DHAVE_R_CETYPE_T=1 -DHAVE_XML_WITH_ZLIB=1 -DHAVE_XML_HAS_FEATURE=1 -DUSE_R=1 -D_R_=1  -DHAVE_VALIDITY=1 -DXML_REF_COUNT_NODES=1  -I. -DLIBXML2=1  -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1642327566152/work=/usr/local/src/conda/r-base-4.1.2 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c DocParse.c -o DocParse.o
x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/in

installing to /opt/conda/lib/R/library/00LOCK-XML/00new/XML/libs
** R
** inst
** byte-compile and prepare package for lazy loading


Creating a generic function for ‘source’ from package ‘base’ in package ‘XML’
in method for ‘xmlAttrsToDataFrame’ with signature ‘"AsIs"’: no definition for class “AsIs”
in method for ‘readKeyValueDB’ with signature ‘"AsIs"’: no definition for class “AsIs”
in method for ‘readSolrDoc’ with signature ‘"AsIs"’: no definition for class “AsIs”


** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (XML)
* installing *source* package ‘snow’ ...
** package ‘snow’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (snow)
* installing *source* package ‘BH’ ...
** package ‘BH’ successfully unpacked and MD5 sums checked
** using staged installation
** inst
** help
*** instal

In [ ]:
%%R
BiocManager::install("DESeq2")

R[write to console]: Bioconductor version 3.14 (BiocManager 1.30.22), R 4.1.2 (2021-11-01)

R[write to console]: Installing package(s) 'DESeq2'

R[write to console]: also installing the dependencies ‘Biostrings’, ‘KEGGREST’, ‘GenomeInfoDb’, ‘XVector’, ‘DelayedArray’, ‘AnnotationDbi’, ‘annotate’, ‘IRanges’, ‘GenomicRanges’, ‘SummarizedExperiment’, ‘BiocParallel’, ‘genefilter’, ‘geneplotter’


R[write to console]: trying URL 'https://bioconductor.org/packages/3.14/bioc/src/contrib/Biostrings_2.62.0.tar.gz'

R[write to console]: Content type 'application/octet-stream'
R[write to console]:  length 12645827 bytes (12.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wri

x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG  -I'/opt/conda/lib/R/library/S4Vectors/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1642327566152/work=/usr/local/src/conda/r-base-4.1.2 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c CompressedAtomicList_utils.c -o CompressedAtomicList_utils.o
x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG  -I'/opt/conda/lib/R/library/S4Vectors/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib   -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/i

make: *** [/opt/conda/lib/R/etc/Makeconf:170: NCList.o] Interrupt
ERROR: compilation failed for package ‘IRanges’
* removing ‘/opt/conda/lib/R/library/IRanges’
* installing *source* package ‘BiocParallel’ ...
** using staged installation


checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for shm_open in -lrt... yes
configure: creating ./config.status


In [ ]:
sampleFiles<- c(
    "SRR391535-output_basename.counts",
    "SRR391536-output_basename.counts",
    "SRR391537-output_basename.counts",
    "SRR391538-output_basename.counts",
    "SRR391539-output_basename.counts",
    "SRR391541-output_basename.counts"
)
sampleNames <- c(
    "Leaf tissue 1",
    "Leaf tissue 2",
    "Leaf tissue 3",
    "Leaf tissue 4",
    "Leaf tissue 5",
    "Leaf tissue 6"
)
sampleCondition <- c(
    "ambient",
    "ambient",
    "elevated",
    "elevated",
    "elevated",
    "ambient"
)
sampleTable <- data.frame(
    sampleName = sampleNames,
    fileName = sampleFiles,
    condition = sampleCondition
)
treatments = c("ambient","elevated")

ddsHTSeq <- DESeqDataSetFromHTSeqCount(
    sampleTable = sampleTable,
    directory = directory,
    design = ~ condition
)
colData(ddsHTSeq)$condition <- factor(colData(ddsHTSeq)$condition, levels = treatments)

### Perform DESeq2 and extract results

In [ ]:
dds <- DESeq(ddsHTSeq)
res <- results(dds)